In [1]:
from langchain_google_vertexai import ChatVertexAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.output_parsers.json import SimpleJsonOutputParser

import os
import json
import re

from typing import Set, List

import random

## Загрузка датасета

In [2]:
PATH = "..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

print(len(data))

83755


## Объявление LLM

### Gemini

In [57]:
PROJECT_ID = 'axial-chemist-425510-p2'
LOCATION = "europe-west4"

# Укажите путь к вашему JSON-файлу с кредами
SERVICE_ACCOUNT_FILE = "../secrets/axial-chemist-425510-p2-b4eb1d622fbe.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_FILE


llm_params = {
    'model_name': 'gemini-2.0-flash-001',
    'project': PROJECT_ID,
    'location': LOCATION
}

llm = ChatVertexAI(**llm_params)

### Т-Pro

In [3]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "../secrets/t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params)

## Скрипт для создания промптов.

In [4]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt

## Функция для тестирования промптов

In [5]:
def test_prompt_on_product(similarity_chain, examples_str, product: dict) -> str:
    res = similarity_chain.invoke({
        "examples": examples_str,
        "problem_title": product["title"],
        "problem_decomposition": json.dumps(product["attributes"], ensure_ascii=False)
    })

    return res

## Вариант извлечения атрибутов №1
Извлекаем только ключи, значения которых есть в товарной позиции.

### Промпты для данного варианта

In [ ]:
SYSTEM_PROMPT = """
Your task is to determine from the dictionary which values ​​are present in the product item.
Examples: 
{examples}

The output should be in the form of an array.

Make sure that:
- Write only the keys of attributes whose values ​​are present in the dictionary.
- Values ​​that are partially present in the product item.
- Important: in the case of a complete absence of the value in the product item, you cannot write the key **not allowed**

**Do not generate any text after the JSON output.**
""".strip()

USER_PROMPT = """
Give an answer for this:
Product item: {problem_title} 
Dictionary: {problem_decomposition}.
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

### Примеры 

In [6]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": '{ "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" }',
        "result": '["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]'
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": '{"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"}',
        "result": '["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]'
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": '{"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"}',
        "result": '["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]'
    }
]

examples_str = "\n".join([f"""
{i+1})Product item: {product["title"]}
Dictionary: {product['attributes']}
Result: {product['result']} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Product item: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Dictionary: { "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия" }
Result: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2)Product item: Сабо женские EVART Ола ЭВА серые (размер 39)
Dictionary: {"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "с

### Запуск модели

In [ ]:
similarity_chain = prompt | llm

for id in idx[:1]:
    res = test_prompt_on_product(similarity_chain, examples_str, data[id])
    

Царга к столу Onix O.M-CS-4 с креплением (белый бриллиант/белый, 1450х18х320 мм)
{
    "Коллекция": "Мебель для персонала ONIX-П",
    "Тип": "царга для стола",
    "Цвет покрытия": "белый , белый бриллиант",
    "Материал": "ЛДСП",
    "Место расположения": "фронтальное",
    "Крепеж в комплекте": "Да",
    "Сборка мебели": "бесплатная сборка",
    "Гарантийный срок": "18 мес",
    "Страна происхождения": "Россия",
    "Высота": "320 мм",
    "Ширина": "1450 мм",
    "Глубина": "18 мм"
}
Ответ: ```json
["Тип", "Цвет покрытия", "Материал", "Крепеж в комплекте", "Высота", "Ширина", "Глубина"]
```
Термогигрометр электронный RGK TH-20 с поверкой (778619)
{
    "Торговая марка": "RGK",
    "Тип": "термогигрометр",
    "ЖК-дисплей": "Да",
    "Память": "нет",
    "Тип питания": "батарейки",
    "Тип измерения": "температура воздуха, влажность воздуха",
    "Гарантийный срок": "12 мес",
    "Комплектация": "батарея питания ААА, руководство по эксплуатации",
    "Количество единиц продаж в тр

In [62]:
similarity_chain = prompt | llm

for id in idx:
    product = data[id]

    res = similarity_chain.invoke({
        "examples": f"{examples_str}",
        "problem_title": product["title"],
        "problem_decomposition": json.dumps(product["attributes"], ensure_ascii=False)
    })

    print(product["title"])
    print(json.dumps(product["attributes"], ensure_ascii=False, indent=4))
    print(f"Ответ: {res.content}")
    
    

Царга к столу Onix O.M-CS-4 с креплением (белый бриллиант/белый, 1450х18х320 мм)
{
    "Коллекция": "Мебель для персонала ONIX-П",
    "Тип": "царга для стола",
    "Цвет покрытия": "белый , белый бриллиант",
    "Материал": "ЛДСП",
    "Место расположения": "фронтальное",
    "Крепеж в комплекте": "Да",
    "Сборка мебели": "бесплатная сборка",
    "Гарантийный срок": "18 мес",
    "Страна происхождения": "Россия",
    "Высота": "320 мм",
    "Ширина": "1450 мм",
    "Глубина": "18 мм"
}
Ответ: ["Тип", "Цвет покрытия", "Материал", "Крепеж в комплекте", "Высота", "Ширина", "Глубина"]
Термогигрометр электронный RGK TH-20 с поверкой (778619)
{
    "Торговая марка": "RGK",
    "Тип": "термогигрометр",
    "ЖК-дисплей": "Да",
    "Память": "нет",
    "Тип питания": "батарейки",
    "Тип измерения": "температура воздуха, влажность воздуха",
    "Гарантийный срок": "12 мес",
    "Комплектация": "батарея питания ААА, руководство по эксплуатации",
    "Количество единиц продаж в транспортной у

## Вариант 2 
Извлекается ключ из атрибутного состава и сопоставляется с частью строки, в которой присутствует упоминание данной товарной позиции.

### Функция для вычисление метрик

In [ ]:
from typing import Dict

def check(ans: List[str], correct_ans: List[str]) -> Dict[str, int]:
    return {
        "correct": len(set(correct_ans).intersection(set(ans))),
        "summary": len(set(ans + correct_ans))
    }

def get_score(prompt, examples_str):

    with open("metrics.json", "r", encoding="utf-8") as file:
        data_metrics = json.load(file)

    batch = [
        {
            "examples": examples_str,
            "problem_title": product["title"],
            "problem_decomposition": product["decomposition"]
        } for product in data_metrics
    ]

    similarity_chain = prompt | llm | SimpleJsonOutputParser()

    res_json = similarity_chain.batch(batch)

    correct, summary = 0, 0
    bad_score = []
    results = []

    for product, res in zip(data_metrics, res_json):
        filtered_res = {key: value for key, value in res.items() if key in product["decomposition"] and value.lower().replace("мм", "") in product["title"].lower()}
        temp = check(list(filtered_res.keys()), product["correct_ans"])
        correct += temp["correct"]
        summary += temp["summary"]

        if temp["summary"] == 0:
            if temp["correct"] == temp["summary"]:
                score = 1
            else:
                score = 0
        else:
            score = (temp["correct"]/temp["summary"])

        if score != 1:
            print(product["title"])
            print(f'correct_ans: {sorted(product["correct_ans"])}')
            if len(res) != len(filtered_res):
                print(f"res: {json.dumps(res, ensure_ascii=False, indent=4)}")
            print(f"filtered_res: {json.dumps(filtered_res, ensure_ascii=False, indent=4)}")
            print(temp)
            print()

        results.append({
            "title": product["title"],
            "correct_ans": product["correct_ans"],
            "categories": product["categories"],
            "ans": filtered_res,
            "score": score
        })   

        if results[-1]["score"] < 0.5:
            bad_score.append([product, filtered_res])

    overall_score = correct/summary
    print(f"correct: {correct}, summary: {summary}, score: {overall_score}")

    return results, correct, summary, overall_score

### Список примеров

In [96]:
examples = [
    {
        "title": "Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)",
        "attributes": '{"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}',
        "result": '{"Торговая марка": "Выбор есть", "Наличие ластика": "с ластиком", "Твердость грифеля": "НВ", "Заточенный": "заточенный", "Профиль карандаша": "шестигранный", "Цвет корпуса": "черный корпус", "Кол-во штук в единице продажи": "12 штук в наборе"}'
    },
    {
        "title": "Сабо женские EVART Ола ЭВА серые (размер 39)",
        "attributes": '{"Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"}',
        "result": '{"Торговая марка": "EVART", "Коллекция": "Ола ЭВА", "Тип": "Сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женские", "Размер": "размер 39"'
    },
    {
        "title": "Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)",
        "attributes": '{"Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"}',
        "result": '{"Торговая марка": "Xerox Colotech +", "Плотность листа бумаги": "250", "Формат листов": "А4", "Количество листов в упаковке": "250 листов"}'
    },
    {
        "title": "Сервер iRU Rock S2208P (2023193)",
        "attributes": '{"Торговая марка": "iRU", "Количество установленных процессоров": "2 шт.", "Максимальное количество процессоров": "2 шт.", "Производитель процессора": "Intel", "Модель чипсета": "C621", "Процессор": "Xeon Silver", "Модель процессора": "4210R", "Число ядер процессора": "10 шт.", "Частота процессора": "2400 мгц", "Тип памяти": "DDR4", "Объём оперативной памяти": "128 Гб", "Количество слотов памяти": "16 шт.", "Максимальное количество оперативной памяти": "4096 гб", "Тип жесткого диска": "SSD", "Интерфейс": "SATA 6 Гбит/с, SAS 12 Гбит/с", "Количество установленных дисков": "1", "Поддерживаемые уровни RAID": "0, 1, 5, 10", "Оптический привод": "нет", "Видеокарта": "Aspeed AST2500", "Форм-фактор блока": "Plug-in module", "Установлено блоков питания": "2", "Максимальное количество блоков питания": "2", "Блок питания (мощность)": "1000 вт", "Сетевой интерфейс": "3xLAN 1000 Мбит/с", "Форм-фактор корпуса": "2U", "Отсеки 3,5\" внешние": "8", "Цвет": "черный", "Операционная система": "отсутствует", "Комплектация": "руководство по эксплуатации", "Гарантийный срок": "36 мес", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Частота оперативной памяти": "2933 МГц", "Объем жесткого диска": "отсутствует Гб", "Объём SSD диска": "500 Гб", "Размер": "647x437x89 мм"}',
        "result": '{"Торговая марка": "iRU"}'
    },
    {
        "title": "Набор гелевых ручек Sketch&Art Uni Write.Glitter 8 цветов (толщина линии 1 мм) (20-0309)",
        "attributes": '{"Торговая марка": "Sketch&Art", "Единица продажи": "набор", "Цвет чернил": "разноцветный", "Количество цветов": "8", "Возможность смены стержня": "нет", "Количество штук в упаковке": "8", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "отсутствует", "Количество единиц продаж в транспортной упаковке": "24", "Страна происхождения": "Китай", "Толщина линии письма": "1 мм"}',
        "result": '{"Торговая марка": "Sketch&Art", "Единица продажи": "набор", "Количество цветов": "8 цветов", "Толщина линии письма": "толщина линии 1"}'
    },
    {
        "title": "Шкаф для документов Linkor 147H112 со стеклом (орех линкольн/белый, 800х398х1835 мм)",
        "attributes": '{"Торговая марка": "Easy To Lead", "Коллекция": "Кабинет Linkor", "Тип": "шкаф", "Цвет покрытия": "белый, орех линкольн", "Количество полок (шт)": "8", "Замок": "Нет", "Материал": "ДСП, стекло", "Материал кромки": "ABS-пластик", "Материал дверей": "стекло, ЛДСП", "Сборка мебели": "бесплатная сборка", "Гарантийный срок": "36 мес", "Страна происхождения": "Беларусь", "Высота": "1835 мм", "Ширина": "800 мм", "Глубина": "398 мм"}',
        "result": '{"Коллекция": "Linkor", "Тип": "шкаф", "Цвет покрытия": "орех линкольн/белый", "Высота": "1835", "Ширина": "800", "Глубина": "398"}'
    },
    {
        'title': 'Папка-планшет с зажимом и крышкой Attache A4 пластиковая черная', 
        'attributes': '{"Торговая марка": "Attache", "Тип папки планшета": "с крышкой", "Формат": "А4", "Расположение зажима": "сверху", "Количество зажимов": "1", "Материал папки": "пластик", "Материал покрытия": "ПВХ", "Защита нижнего края папки": "отсутствует", "Цвет": "черный", "Прозрачный": "Нет", "Фактура": "песок", "С зажимом": "да", "Наличие кармана на внутренней обложке папки": "Нет", "Держатель для ручки": "Нет", "Возможность крепления к стене": "Нет", "Вместимость": "75 листов", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "отсутствует", "Количество единиц продаж в транспортной упаковке": "20", "Страна происхождения": "Россия", "Размер": "230x310 мм", "Толщина материала": "1.2 мм"}', 
        'result': '{"Торговая марка": "Attache", "Тип папки планшета": "крышкой", "Формат": "А4", "Материал папки": "пластиковая", "Цвет": "черный", "С зажимом": "с зажимом"}'
    }

]

examples_str = "\n".join([f"""
{i+1})Product: {product["title"]}
Dictionary of attributes: {product['attributes']}
Result: {product['result']} 
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Product: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе)
Dictionary of attributes: {"Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"}
Result: {"Торговая марка": "Выбор есть", "Наличие ластика": "с ластиком", "Твердость грифеля": "НВ", "Заточенный": "заточенный", "Профиль карандаша": "шестигранный", "Цвет корпуса": "черный корпус", "Кол-во штук в единице продажи": "12 штук в наборе"}
2)Product: Сабо женские EVART Ола ЭВА серые (размер 39)
Dictionary of attributes: {"Торговая марка": "Evart", "Коллекция": "Сабо 

### 1 вариант промпта
Получаемые метрики около 0.8-0.81

In [74]:
SYSTEM_PROMPT = """
Write a presence-correspondence dictionary that will describe the relationship between a key and a fragment of a product item.
Examples:
{examples}

The result of the work must be in the form of JSON.

Pay attention to the strict implementation of these rules:
- The dictionary must consist only of dictionary keys, and the values ​​must be only fragments of the product item.
- Do not write those keys for which there is no match.
- Values ​​should not intersect between other values ​​in your answer.

**Do not generate any text after the JSON output.**
""".strip()

USER_PROMPT = """
Give an answer for this:
Product item: {problem_title} 
Dictionary: {problem_decomposition}.
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [75]:
get_score(prompt, examples_str)

Тетрадь школьная Комус Класс (№1 School) голубая А5 24 листа в клетку (10 штук в упаковке)
correct_ans: ['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов', 'Цвет линовки', 'Цвет обложки']
filtered_res: {
    "Торговая марка": "Комус Класс",
    "Формат листов": "А5",
    "Количество листов": "24 листа",
    "Вид линовки": "в клетку",
    "Цвет обложки": "голубая",
    "Количество штук в упаковке": "10 штук в упаковке"
}
{'correct': 6, 'summary': 7}

Тетрадь школьная Альт Военный паттерн А5 24 листа в линейку (10 штук в упаковке)
correct_ans: ['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов']
res: {
    "Торговая марка": "Альт",
    "Формат листов": "А5",
    "Количество листов": "24 листа",
    "Вид линовки": "в линейку",
    "Цвет обложки": "Военный паттерн",
    "Количество штук в упаковке": "10 штук в наборе"
}
filtered_res: {
    "Торговая марка": "Альт",
    "Формат листов": "А5",


([{'title': 'Тетрадь школьная Комус Класс (№1 School) голубая А5 24 листа в клетку (10 штук в упаковке)',
   'correct_ans': ['Торговая марка',
    'Формат листов',
    'Количество листов',
    'Вид линовки',
    'Цвет обложки',
    'Цвет линовки',
    'Количество штук в упаковке'],
   'categories': ['Школьные тетради (12-24 листов)',
    'Школьные блокноты и тетради'],
   'ans': {'Торговая марка': 'Комус Класс',
    'Формат листов': 'А5',
    'Количество листов': '24 листа',
    'Вид линовки': 'в клетку',
    'Цвет обложки': 'голубая',
    'Количество штук в упаковке': '10 штук в упаковке'},
   'score': 0.8571428571428571},
  {'title': 'Смазка CET CET2828',
   'correct_ans': ['Торговая марка', 'Тип'],
   'categories': ['Запасные части (ЗИП) для принтеров'],
   'ans': {'Торговая марка': 'CET', 'Тип': 'смазка'},
   'score': 1.0},
  {'title': 'Кулер для процессора DeepCool ICE EDGE Mini RET',
   'correct_ans': ['Торговая марка'],
   'categories': ['Кулеры для процессора'],
   'ans': {'Тор

### 2 Вариант промпта
Метрики около 0.79-0.8

In [76]:
SYSTEM_PROMPT = """
Создай словарь соответствий, который описывает связь между ключом (атрибутом) и фрагментом текста из описания товара, соответствующим этому атрибуту.

Примеры:
{examples}

Результат работы должен быть в формате JSON.

Обрати внимание на строгое соблюдение следующих правил:
- Ключами в словаре должны быть ключи из предоставленного словаря атрибутов.
- Значениями в словаре должны быть только фрагменты текста из описания товара, которые наиболее точно соответствуют значению ключа.
- Не включай в словарь ключи, для которых не удалось найти соответствующий фрагмент в описании товара.
- Значения в словаре не должны пересекаться между собой.

Не генерируй никакого текста после JSON.
""".strip()

USER_PROMPT = """
Для следующего товара:
Описание товара: {problem_title}
Словарь атрибутов: {problem_decomposition}

создай словарь соответствий.
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [77]:
get_score(prompt, examples_str)

Тетрадь школьная Альт Военный паттерн А5 24 листа в линейку (10 штук в упаковке)
correct_ans: ['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов']
filtered_res: {
    "Торговая марка": "Альт",
    "Формат листов": "А5",
    "Количество листов": "24 листа",
    "Вид линовки": "в линейку",
    "Цвет обложки": "Военный паттерн",
    "Количество штук в упаковке": "10 штук в упаковке"
}
{'correct': 5, 'summary': 6}

Тумба приставная Metal System Style Б.ТП-1 (акация лорка, 412x720x750 мм)
correct_ans: ['Вид', 'Высота', 'Глубина', 'Коллекция', 'Цвет покрытия', 'Ширина']
res: {
    "Коллекция": "Metal System Style",
    "Вид": "тумба",
    "Цвет покрытия": "акация лорка",
    "Глубина": "720 мм",
    "Высота": "750 мм",
    "Ширина": "412 мм"
}
filtered_res: {
    "Коллекция": "Metal System Style",
    "Вид": "тумба",
    "Цвет покрытия": "акация лорка",
    "Высота": "750 мм"
}
{'correct': 4, 'summary': 6}

Пуф Куба складной с ящиком для хране

([{'title': 'Тетрадь школьная Комус Класс (№1 School) голубая А5 24 листа в клетку (10 штук в упаковке)',
   'correct_ans': ['Торговая марка',
    'Формат листов',
    'Количество листов',
    'Вид линовки',
    'Цвет обложки',
    'Цвет линовки',
    'Количество штук в упаковке'],
   'categories': ['Школьные тетради (12-24 листов)',
    'Школьные блокноты и тетради'],
   'ans': {'Торговая марка': 'Комус Класс',
    'Формат листов': 'А5',
    'Количество листов': '24 листа',
    'Вид линовки': 'в клетку',
    'Цвет линовки': 'голубая',
    'Цвет обложки': 'голубая',
    'Количество штук в упаковке': '10 штук в упаковке'},
   'score': 1.0},
  {'title': 'Смазка CET CET2828',
   'correct_ans': ['Торговая марка', 'Тип'],
   'categories': ['Запасные части (ЗИП) для принтеров'],
   'ans': {'Торговая марка': 'CET', 'Тип': 'смазка'},
   'score': 1.0},
  {'title': 'Кулер для процессора DeepCool ICE EDGE Mini RET',
   'correct_ans': ['Торговая марка'],
   'categories': ['Кулеры для процессора'],

### 3 вариант промпта
Получаемые метрики около 0.88-0.89

In [97]:
SYSTEM_PROMPT = """
Create a dictionary of correspondences that describes the relationship between the key (attribute) and the text fragment from the product description corresponding to this attribute.

Examples:
{examples}

The result must be in JSON format.

Pay attention to the strict observance of the following rules:
- The keys in the dictionary **must be keys** from the provided dictionary of attributes.
- The values in the dictionary should be only fragments of text from the product description that **exactly**** or **partially** correspond to the value of the key.
- The values in the dictionary must fully or partially match the value in the dictionary of attributes.
- Do not include keys in the dictionary for which it was not possible to find the corresponding fragment in the product description.
- Double-check whether the value in your answer is in the product name.
- If dimensions are present, then **do not write ** units.


Do not generate any text after JSON.
""".strip()

USER_PROMPT = """
Create a dictionary of matches:
Product: {problem_title}
Dictionary of attributes: {problem_decomposition}
""".strip()


prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [98]:
results, correct, summary, overall_score = get_score(prompt, examples_str)

Тетрадь школьная Альт Военный паттерн А5 24 листа в линейку (10 штук в упаковке)
correct_ans: ['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов']
filtered_res: {
    "Торговая марка": "Альт",
    "Формат листов": "А5",
    "Количество листов": "24 листа",
    "Вид линовки": "в линейку",
    "Цвет обложки": "Военный паттерн",
    "Количество штук в упаковке": "10 штук в упаковке"
}
{'correct': 5, 'summary': 6}

Папка-планшет с зажимом Attache A4 пластиковая черная
correct_ans: ['Материал папки', 'С зажимом', 'Торговая марка', 'Формат', 'Цвет']
res: {
    "Торговая марка": "Attache",
    "Формат": "А4",
    "Материал папки": "пластиковая",
    "Цвет": "черная",
    "С зажимом": "с зажимом"
}
filtered_res: {
    "Торговая марка": "Attache",
    "Материал папки": "пластиковая",
    "Цвет": "черная",
    "С зажимом": "с зажимом"
}
{'correct': 4, 'summary': 5}

Папка-конверт на молнии Attache А5 бесцветная 230 мкм (2 штуки в упаковке)
correct

#### Анализ результатов 3 промпта

In [99]:
# Величина ошибки
not_correct_match_products = [product for product in results if product["score"] != 1]
checked = [check(list(product["ans"].keys()), product["correct_ans"]) for product in not_correct_match_products]
error_values = [d["summary"] - d["correct"] for d in checked]

print(error_values)
print(f"Средняя величина ошибки: {sum(error_values) / len(error_values)}")

[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 5, 1, 3, 1, 1, 3, 1, 1, 2, 1, 1, 3, 4, 1, 1, 1, 2, 1]
Средняя величина ошибки: 1.5128205128205128


In [100]:
from collections import Counter
# Категории неправильных ответов

c = Counter()

for product in not_correct_match_products:
    c.update(product["categories"])

c


Counter({'Папки-планшеты': 4,
         'Офисные защитные экраны на струбцинах': 4,
         'Рабочая непромокаемая спецодежда': 4,
         'Наборы посуды для приготовления': 4,
         'Кастрюли, сотейники, котлы, мантоварки': 4,
         'Ручки гелевые': 3,
         'Ручки': 3,
         'Кулеры для процессора': 3,
         'Профессиональные акустические системы': 3,
         'Конференц - связь': 3,
         'Цветные карандаши, фломастеры': 3,
         'Карандаши цветные': 3,
         'Шкафы для одежды': 2,
         'Магнитно-маркерные доски': 2,
         'Элементы бенч-систем': 2,
         'Тетради и блокноты': 1,
         'Школьные тетради (12-24 листов)': 1,
         'Школьные блокноты и тетради': 1,
         'Папки-конверты': 1,
         'Пуфы и бескаркасная мебель': 1,
         'Пуфы': 1,
         'Ламинаторы': 1,
         'Стеновые панели для кухни': 1})

In [104]:
import plotly.express as px

keys = [item for item in c.keys()]
values = [item for item in c.values()]

# Создание DataFrame (необязательно, но удобно для plotly)
import pandas as pd
df = pd.DataFrame({'Атрибут': keys, 'Количество товаров': values})

# Создание графика
fig = px.bar(
    df,
    x='Атрибут',
    y='Количество товаров',
    title=f'График распределения категорий в которых были произведены ошибки',
    color='Количество товаров',  # Цветовая шкала
    text='Количество товаров'    # Отображение значений над столбцами
)

# Настройка визуализации
fig.update_traces(texttemplate='%{text:.2s}', textposition='auto')
fig.update_layout(showlegend=False)

# Отображение графика
fig.show()

# Вариант 3

In [42]:
import re
from fuzzywuzzy import fuzz

def match_title_to_decomposition(title, decomposition, text_threshold=60):
    matched_keys = []

    # 1. Поиск текстовых совпадений (Fuzzy Matching)
    for key, value in decomposition.items():
        score = fuzz.partial_ratio(value.lower(), title.lower())
        if score >= text_threshold:
            matched_keys.append(key)

    # 2. Извлечение размеров из title
    size_match = re.search(r'(\d+)х(\d+)х(\d+)', title)
    if size_match:
        extracted_sizes = {"Ширина": size_match.group(1), "Глубина": size_match.group(2), "Высота": size_match.group(3)}
    else:
        extracted_sizes = {}

    # 3. Сопоставление размеров с decomposition
    for key, extracted_value in extracted_sizes.items():
        if key in decomposition:
            decomposed_value = decomposition[key].split()[0]
            if extracted_value == decomposed_value:
                matched_keys.append(key)

    return matched_keys

In [ ]:
with open("metrics.json", "r", encoding="utf-8") as file:
    data_metrics = json.load(file)

correct, summary = 0, 0
bad_score = []
results = []

for product in data_metrics:
    filtered_res = match_title_to_decomposition(product["title"], product["decomposition"], text_threshold=95)
    temp = check(filtered_res, product["correct_ans"])
    correct += temp["correct"]
    summary += temp["summary"]

    if temp["summary"] == 0:
        if temp["correct"] == temp["summary"]:
            score = 1
        else:
            score = 0
    else:
        score = (temp["correct"]/temp["summary"])

    if score != 1:
        print(product["title"])
        print(f'correct_ans: {sorted(product["correct_ans"])}')
        # if len(res) != len(filtered_res):
        #     print(f"res: {json.dumps(res, ensure_ascii=False, indent=4)}")
        print(f"filtered_res: {json.dumps({key: product['decomposition'][key] for key in filtered_res}, ensure_ascii=False, indent=4)}")
        print(temp)
        print()

    results.append({
        "title": product["title"],
        "correct_ans": product["correct_ans"],
        "ans": filtered_res,
        "score": score
    }) 

overall_score = correct/summary
print(f"correct: {correct}, summary: {summary}, score: {overall_score}")  

Тетрадь школьная Комус Класс (№1 School) голубая А5 24 листа в клетку (10 штук в упаковке)
correct_ans: ['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов', 'Цвет линовки', 'Цвет обложки']
filtered_res: {
    "Торговая марка": "Комус Класс",
    "Формат листов": "А5",
    "Количество штук в упаковке": "10"
}
{'correct': 3, 'summary': 7}

Термистор CET CET531003
correct_ans: ['Тип', 'Торговая марка']
filtered_res: {
    "Торговая марка": "Cet",
    "Тип": "термистор",
    "Количество единиц продаж в транспортной упаковке": "1"
}
{'correct': 2, 'summary': 3}

Тетрадь школьная Альт Военный паттерн А5 24 листа в линейку (10 штук в упаковке)
correct_ans: ['Вид линовки', 'Количество листов', 'Количество штук в упаковке', 'Торговая марка', 'Формат листов']
filtered_res: {
    "Торговая марка": "Альт",
    "Формат листов": "А5"
}
{'correct': 2, 'summary': 5}

Тумба приставная Metal System Style Б.ТП-1 (акация лорка, 412x720x750 мм)
correct_ans: 